In [17]:
import cv2
import torch
from PIL import Image
from mss import mss
import numpy as np

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.4
model.classes = [2, 3, 4, 5, 6]  
sct = mss()

xb_min = 290+200
xb_max = 1650-200
yb_min = 240
yb_max = 900
while 1:
    w, h = 1920, 1080
    monitor = {'top': 0, 'left': 0, 'width': w, 'height': h}
    img = Image.frombytes('RGB', (w, h), sct.grab(monitor).rgb)
    screen = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    # set the model use the screen
    result = model(screen, size=1024)
    pd = result.pandas().xyxy[0]
    
    predicted_image = result.render()[0]

    for index, row in pd.iterrows():
        if True:
            xmin = row['xmin']
            ymin = row['ymin']
            xmax = row['xmax']
            ymax = row['ymax']

            mid_x = (xmax-xmin)/2 + xmin
            mid_y = (ymax-ymin)/2 + ymin
            coef = round((xmax-xmin)/(xb_max-xb_min) + (ymax-ymin)/(xb_max-xb_min), 3)
            if (xb_min <= mid_x <=  xb_max) and (yb_min <= mid_y <= yb_max) and coef >= 0.39:
                cv2.putText(
                    img = predicted_image,
                    text = f"WARNING!",
                    org = (int(mid_x), int(mid_y)),
                    fontFace = cv2.FONT_HERSHEY_DUPLEX,
                    fontScale = 2,
                    color = (0, 10, 240),
                    thickness = 3
                    )
    # predicted_image = cv2.putText(
    #         img = predicted_image,
    #         text = "Hello World",
    #         org = (200, 200),
    #         fontFace = cv2.FONT_HERSHEY_DUPLEX,
    #         fontScale = 3.0,
    #         color = (125, 246, 55),
    #         thickness = 3
    #         )
    cv2.imshow('Screen', predicted_image)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

Using cache found in C:\Users\cheek/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-27 Python-3.8.0 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
"""

model.conf = 0.25  # NMS confidence threshold
      iou = 0.45  # NMS IoU threshold
      agnostic = False  # NMS class-agnostic
      multi_label = False  # NMS multiple labels per box
      classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
      max_det = 1000  # maximum number of detections per image
      amp = False  # Automatic Mixed Precision (AMP) inference

results = model(im, size=320)  # custom inference size

"""

In [ ]:
import cv2
import torch
from PIL import Image
from mss import mss
import numpy as np
from torchvision.models.detection import  fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
from torchvision.io.image import read_image
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

plt.rcParams["savefig.bbox"] = 'tight'


def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

weights = FasterRCNN_ResNet50_FPN_Weights.COCO_V1
model = fasterrcnn_resnet50_fpn(weights=weights)
model.eval()
sct = mss()

transform = transforms.Compose([
    transforms.ToTensor()
])




while 1:
    w, h = 1920, 1080
    monitor = {'top': 0, 'left': 0, 'width': w, 'height': h}
    img = Image.frombytes('RGB', (w, h), sct.grab(monitor).rgb)
    screen = cv2.cvtColor(np.array(screen), cv2.COLOR_RGB2BGR)
    screen = transform(screen)
    output = model(screen)
    print(output)
    # if cv2.waitKey(25) & 0xFF == ord('q'):
    #     cv2.destroyAllWindows()
    #     break 

In [ ]:
import torchvision.transforms as transforms
import cv2
import numpy
import numpy as np
import torchvision
import numpy
import torch
import argparse
import cv2
from PIL import Image
from mss import mss



coco_names = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

COLORS = np.random.uniform(0, 255, size=(len(coco_names), 3))
sct = mss()
transform = transforms.Compose([
    transforms.ToTensor()
])


def predict(image, model, device, detection_threshold):
    # transform the image to tensor
    image = transform(image).to(device)
    image = image.unsqueeze(0) # add a batch dimension
    outputs = model(image) # get the predictions on the image
    # print the results individually
    # print(f"BOXES: {outputs[0]['boxes']}")
    # print(f"LABELS: {outputs[0]['labels']}")
    # print(f"SCORES: {outputs[0]['scores']}")
    # get all the predicited class names
    pred_classes = [coco_names[i] for i in outputs[0]['labels'].cpu().numpy()]
    # get score for all the predicted objects
    pred_scores = outputs[0]['scores'].detach().cpu().numpy()
    # get all the predicted bounding boxes
    pred_bboxes = outputs[0]['boxes'].detach().cpu().numpy()
    # get boxes above the threshold score
    boxes = pred_bboxes[pred_scores >= detection_threshold].astype(np.int32)
    return boxes, pred_classes, outputs[0]['labels']

def draw_boxes(boxes, classes, labels, image):
    # read the image with OpenCV
    image = cv2.cvtColor(np.asarray(image), cv2.COLOR_BGR2RGB)
    for i, box in enumerate(boxes):
        color = COLORS[labels[i]]
        cv2.rectangle(
            image,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color, 2
        )
        cv2.putText(image, classes[i], (int(box[0]), int(box[1]-5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, 
                    lineType=cv2.LINE_AA)

    return image

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, min_size=1024)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval().to(device)


while 1:
    w, h = 1920, 1080
    monitor = {'top': 0, 'left': 0, 'width': w, 'height': h}
    image = Image.frombytes('RGB', (w, h), sct.grab(monitor).rgb)
    boxes, classes, labels = predict(image, model, device, 0.8)
    image = draw_boxes(boxes, classes, labels, image)
    cv2.imshow('Image', image)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break